# Capstone Project - Final Report

### Title:    Tourism Recommender System

## 1. Introduction/Business Problem

The Idea of the project is to recommend cities to some user to visit, based on previous cities he visited before. 
 
Recommender Systems are data science tools, which attempt to predict items out of large pool a user may highly likely be interested in, and to suggest him the best one. They also support systems helping users to find and/ or to make choices about items that matches their preferences and interests.

In the tourism field, recommender systems aim to match the characteristics of tourism and leisure resources or attractions with the user needs.

By Defining cities characteristics and using user preferences, the recommender system can suggest some new cities for this user, cities that are similar to his needs.

The audience of the project are tourists. That is, by helping them choosing cities they like to visit.

## 2. Data

There is to types of data sources will be used in this project.

The first one is Foursquare Location Data.  
The second one is Postal Codes with the names of neighbourhoods of the major cities in the world.

### 2.1. Foursquare Location Data

Foursquare is a technology company that built a massive dataset of location data. What is interesting about Foursquare is that they were very smart about building their dataset. They actually crowd-sourced their data and had people use their app to build their dataset and add venues and complete any missing information they had in their dataset. Currently its location data is the most comprehensive out there, and quite accurate that it powers location data for many popular services like Apple Maps, Uber, Snapchat, Twitter and many others, and is currently being used by over 100,000 developers, and this number is only growing. 


## 2.2. Postal Codes Data

The Postal Codes Data for most of countries is available at AggData Website (https://www.aggdata.com).
I will download the data of three countries: United States, France and Italy. And then, I will get the major cities for each country.

The Postal Codes from AggData are able to be downloaded as csv files. The data contains also the location of the neighbourhoods which makes it very useful to use it along side with Forusquare API

An Example of Postal Codes Data


![title](example.jpg)